In [ ]:
# Install required libraries
!pip install streamlit
!pip install pyngrok
!pip install openai
!pip install diffusers
!pip install transformers scipy ftfy accelerate
!pip install git+https://github.com/huggingface/diffusers
!pip install safetensors
!pip install langchain
!pip3 install python-dotenv
!pip install google-search-results
!pip install Pillow
%pip install stability-sdk

In [ ]:
pip install invisible_watermark safetensors

In [119]:
%%writefile app.py
# Import necessary libraries
import openai
from diffusers import DiffusionPipeline, StableDiffusionXLPipeline
import streamlit as st
from PIL import Image
import torch
import urllib
from langchain.agents import AgentType, initialize_agent, load_tools
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import time
from diffusers import DiffusionPipeline, StableDiffusionXLPipeline
import io
import warnings
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation
import IPython.display as display

# Set API keys and configurations
export OPENAI_API_KEY='your-openai-api-key'
export SERPAPI_API_KEY='your-serpapi-api-key'
export STABILITY_HOST='grpc.stability.ai:443'
export STABILITY_KEY='your-stability-key'

# Configure Streamlit app
st.set_page_config(
    page_title="AI Blog Generator",
    page_icon=":pushpin:",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Function to generate images using different engines
def generate_images(prompt, select_model):

    if select_model == "segmind/SSD-1B":
        device = "cuda"
        pipe = StableDiffusionXLPipeline.from_pretrained("segmind/SSD-1B", torch_dtype=torch.float16,
                                                         use_safetensors=True, variant="fp16")
        pipe.to("cuda")
        neg_prompt = "ugly, blurry, poor quality"
        return pipe(prompt=prompt, negative_prompt=neg_prompt).images[0]

    elif select_model == "stable-diffusion-xl-1024-v1-0":
        stability_api = client.StabilityInference(
            key=os.environ['STABILITY_KEY'],
            verbose=True,
            engine="stable-diffusion-xl-1024-v1-0",
        )
        answers = stability_api.generate(
            prompt=prompt,
            seed=4254978046, steps=50, cfg_scale=8.0, samples=1,
            sampler=generation.SAMPLER_K_DPMPP_2M
        )
        for resp in answers:
            for artifact in resp.artifacts:
                if artifact.type == generation.ARTIFACT_IMAGE:
                    img = Image.open(io.BytesIO(artifact.binary))
                    return img


# Streamlit UI styling
st.markdown(
    """
    <style>
        .sidebar .sidebar-content {
            background-color: #31bd56;
            color: #F1D700;
        }
        .sidebar .stButton {
            background-color: #F7D700;
            color: #233142;
        }
        .sidebar .stSelectbox {
            background-color: #FFD700;
            color: #2D3742;
        }
        .widget-title {
            color: #2D3142;
        }
        .main .wrapper {
            background-color: #F4F4F4;
        }
        .stMarkdown {
            color: #d1d6e8;
            font-size: 18px;
        }
        .stHeader {
            color: #2A3142;
            font-size: 30px;
        }
        .stSubheader {
            color: #2D3142;
            font-size: 24px;
        }
    </style>
    """,
    unsafe_allow_html=True
)

# Streamlit App

with st.sidebar:
    st.title("AI Blog Generator")
    num_of_keywords = st.text_input("Enter the number of Keywords:")
    topic = st.text_input("Enter the topic title:")
    select_engine = st.selectbox("Select Model", ["segmind/SSD-1B", "stable-diffusion-xl-1024-v1-0"])
    button = st.button("Generate Blog")

if button:
    # Keywords
    prompt_template_name = PromptTemplate(
        input_variables=['Topic', 'NumOfKeywords'],
        template=(
            """Provide the most common {NumOfKeywords} keywords related to the topic "{Topic}".
              Please just list the keywords in point form as follows:
              1.
              2.
              3.
            """
        )
    )

    p = prompt_template_name.format(Topic=topic, NumOfKeywords=num_of_keywords)

    llm = ChatOpenAI(temperature=0.5)

    # Initialize agent
    tools = load_tools(["serpapi", "llm-math", "requests_all"], llm=llm)
    agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True, handle_parsing_errors=True)
    result = agent.run(p)
    splited_result = result.split("\n")
    st.header(f"Blog related to {topic}")
    with st.sidebar:
         st.markdown(f"""
                ##Keywords:
{result}
                """)

    for i, Keyword in enumerate(splited_result):
        if i != 1:
            st.subheader(f"Keyword: {Keyword}")
            prompt_template_name = PromptTemplate(
                input_variables=['keyword', 'Topic'],
                template=(
                    """Generate a paragraph of up to 80 words about {keyword} in the context of {Topic}."""
                )
            )
            time.sleep(20)
            p = prompt_template_name.format(keyword=Keyword, Topic=topic)
            paragraph_result = agent.run(p)
            st.markdown(f'<div style="margin-top: 10px; margin-bottom: 10px; font-size: 20px;text-align: justify;">{paragraph_result}</div>', unsafe_allow_html=True)

            prompt_template_name = PromptTemplate(
                input_variables=['paragraph_result'],
                template=(
                    """Generate the detailed image prompt that conveying the following text:

                      {paragraph_result}

                      Follow this format: [type of art], [subject or topic], [action or activity], [aesthetic details, lighting, and styles], [colors], [--ar aspect ratio]
                      """
                )
            )
            time.sleep(10)
            p = prompt_template_name.format(paragraph_result=paragraph_result)
            agent_for_image_prompt_1 = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors=True)
            image_prompt = agent_for_image_prompt_1.run(p)
            image = generate_images(prompt=image_prompt, select_engine=select_engine)
            st.image(image, caption=f"Generated Image for {Keyword}", use_column_width=True)
            with st.expander("See prompt"):
                 st.markdown(f'<div style="margin-top: 10px; margin-bottom: 10px; font-size: 16px;text-align: justify;">{image_prompt}</div>', unsafe_allow_html=True)

        else:
            col1, col2 = st.columns(2)

            with col1:
                st.subheader(f"Keyword: {Keyword}")
                prompt_template_name = PromptTemplate(
                    input_variables=['keyword', 'Topic'],
                    template=(
                        """Generate a paragraph of up to 100 words about {keyword} in the context of {Topic}."""
                    )
                )
                time.sleep(10)
                p = prompt_template_name.format(keyword=Keyword, Topic=topic)
                paragraph_result = agent.run(p)
                st.markdown(f'<div style="margin-top: 10px; margin-bottom: 10px; font-size: 18px; text-align: justify;">{paragraph_result}</div>', unsafe_allow_html=True)

            with col2:
                prompt_template_name = PromptTemplate(
                    input_variables=['paragraph_result'],
                    template=(
                        """Generate the detailed image prompt that conveying the following text:

                          {paragraph_result}

                          Follow this format: [type of art], [subject or topic], [action or activity], [aesthetic details, lighting, and styles], [colors], [--ar aspect ratio]
                          """
                    )
                )
                time.sleep(10)
                p = prompt_template_name.format(paragraph_result=paragraph_result)
                agent_for_image_prompt_2 = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors=True)
                image_prompt = agent_for_image_prompt_2.run(p)
                image = generate_images(prompt=image_prompt, select_engine=select_engine)
                st.image(image, caption=f"Generated Image for {Keyword}", use_column_width=True)
                with st.expander("See prompt"):
                     st.markdown(f'<div style="margin-top: 10px; margin-bottom: 10px; font-size: 16px;text-align: justify;">{image_prompt}</div>', unsafe_allow_html=True)


Overwriting app.py


In [5]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 3.102s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [125]:
!streamlit run app.py &>/content/logs.txt &

In [121]:

import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))


Password/Enpoint IP for localtunnel is: 34.123.190.41


In [126]:
!npx localtunnel --port 8501

npx: installed 22 in 1.72s
your url is: https://neat-mugs-cheer.loca.lt
^C
